In [1]:
!pwd
import pyrosetta
import os
import glob
pyrosetta.init("-mute all")
sfxn = pyrosetta.rosetta.core.scoring.ScoreFunction()
sfxn.set_weight(pyrosetta.rosetta.core.scoring.fa_rep, 1)

/mnt/home/flop/switch/5thround/extend
PyRosetta-4 2022 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2022.16+release.839b00f59d92ff5270c37073376bd37d27064c7e 2022-04-19T17:48:30] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [2]:
def range_CA_align(pose1, pose2, start1, end1, start2, end2):
#modified from Adam's function (pose 1 mobile)
    pose1_residue_selection = range(start1,end1)
    pose2_residue_selection = range(start2,end2)
    
    assert len(pose1_residue_selection) == len(pose2_residue_selection)

    pose1_coordinates = pyrosetta.rosetta.utility.vector1_numeric_xyzVector_double_t()
    pose2_coordinates = pyrosetta.rosetta.utility.vector1_numeric_xyzVector_double_t()

    for pose1_residue_index, pose2_residue_index in zip(pose1_residue_selection, pose2_residue_selection):
        pose1_coordinates.append(pose1.residues[pose1_residue_index].xyz('CA'))
        pose2_coordinates.append(pose2.residues[pose2_residue_index].xyz('CA'))

    rotation_matrix = pyrosetta.rosetta.numeric.xyzMatrix_double_t()
    pose1_center = pyrosetta.rosetta.numeric.xyzVector_double_t()
    pose2_center = pyrosetta.rosetta.numeric.xyzVector_double_t()

    pyrosetta.rosetta.protocols.toolbox.superposition_transform(pose1_coordinates,
                                                                pose2_coordinates,
                                                                rotation_matrix,
                                                                pose1_center,
                                                                pose2_center)

    pyrosetta.rosetta.protocols.toolbox.apply_superposition_transform(pose1,
                                                                      rotation_matrix,
                                                                      pose1_center,
                                                                      pose2_center)
                     
def clash_check(pose):
    all_gly = pose.clone()
    true_sel = pyrosetta.rosetta.core.select.residue_selector.TrueResidueSelector()
    true_x = true_sel.apply(all_gly)
    pyrosetta.rosetta.protocols.toolbox.pose_manipulation.repack_these_residues(true_x, all_gly, sfxn, False, "G" )
    score = sfxn(all_gly)
    return score

def helix_dict(pose):
    #pose = pyrosetta.io.pose_from_pdb(filename)
    ss = pyrosetta.rosetta.core.scoring.dssp.Dssp(pose)
    hlx_dict = {}
    n = 1
    for i in range(1,len(pose.sequence())):
        if (ss.get_dssp_secstruct(i) == "H") & (ss.get_dssp_secstruct(i-1) !="H"):
            hlx_dict[n] = [i]
        if (ss.get_dssp_secstruct(i) == "H") & (ss.get_dssp_secstruct(i+1) !="H"):
            hlx_dict[n].append(i)
            n +=1
    return hlx_dict

def load_pose(name):
    if type(name) == str:
        pose = pyrosetta.io.pose_from_pdb(name)
    else:
        try:
            pose = name.clone()
        except:
            print("input must be a pose or a path to a pose")
    return pose


def swap_ab(name):
    pose_ab = load_pose(name)
    pose_ba = pose_ab.split_by_chain(2)
    pyrosetta.rosetta.core.pose.append_pose_to_pose(pose_ba,pose_ab.split_by_chain(1),1)
    return pose_ba
# swap_ab: normally assume that oligomer is chain A and heterodimer is chain B. If the other way round ,then do swap_ab

def extend_dhr(pose1,pose2, keep = "c", h1n = "last", h2n = "last", h1_c = 3, h2_c = 4):
    if type(pose1) == str:
        pose_n = pyrosetta.io.pose_from_pdb(pose1).split_by_chain(1)
    else:
        pose_n = pose1.split_by_chain(1)
    if type(pose2) == str:
        pose_c = pyrosetta.io.pose_from_pdb(pose2).split_by_chain(1)
    else:
        pose_c = pose2.split_by_chain(1)
    hdn = helix_dict(pose_n)
    hdc = helix_dict(pose_c)
    if (h1n == "last") or (h1n == -2):
        h1_n = list(hdn)[-2]
    else:
        h1_n = h1n
    if (h2n == "last") or (h2n == -1):
        h2_n = list(hdn)[-1]
    else:
        h2_n = h2n
    copy_c = pose_c.clone()
    newpose = pyrosetta.rosetta.core.pose.Pose()
    start_c = hdc[h1_c][0]
    end_c = hdc[h2_c][1]
    start_n = hdn[h1_n][0]
    end_n = hdn[h2_n][1]
    if (end_n - start_n) != (end_c - start_c):
        #print(end_n - start_n)
        #print(end_c - start_c)
        if keep == "n":
            start_c = end_c - (end_n - start_n)
        elif keep == "c":
            end_n = start_n + (end_c - start_c) 
        else:
            print("keep must be n or c")
        while (start_c < 1) or (start_n < 1):
            start_c +=1
            start_n +=1
    range_CA_align(copy_c, pose_n, start_c, end_c, start_n, end_n)
    if keep == "n":
        cut1 = end_n 
        cut2 = end_c         
    elif keep == "c":
        cut1 = start_n 
        cut2 = start_c 
    else:
        print("keep must be n or c")
    #print(cut1,cut2)
    for i in range(1,cut1):
        newpose.append_residue_by_bond(pose_n.residue(i))
    for i in range(cut2,len(pose_c.sequence())+1):
        newpose.append_residue_by_bond(copy_c.residue(i))
    offset = (len(newpose.sequence())-len(pose_c.sequence()))    
    
    selx = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector("IntX")
    sely = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector("IntY")
    int_n = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(selx,sely).apply(pose_n)
    int_c = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(selx,sely).apply(pose_c)
    for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_n):
        newpose.replace_residue(i,pose_n.residue(i),1)
    for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_c):
        newpose.replace_residue(i+offset,pose_c.residue(i),1)
    info = pyrosetta.rosetta.core.pose.PDBInfo(newpose,1)
    for label in ["IntX","IntY"]:
        sel = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector(label)
        int_n = sel.apply(pose_n)
        int_c = sel.apply(pose_c)
        for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_n):            
            info.add_reslabel(i,label)
        for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_c):            
            info.add_reslabel(i+offset,label)
        newpose.pdb_info(info)
    return newpose
    
    
def oligomer_align(oligomer,monomer,const_term = "n", mono_chains = [1]):
    oligo = load_pose(oligomer)
    mono_full = load_pose(monomer)
    if mono_chains == [1]:
        mono = mono_full
    else:
        mono = pyrosetta.rosetta.core.pose.Pose()
        for i in mono_chains:
            pyrosetta.rosetta.core.pose.append_pose_to_pose(mono,mono_full.split_by_chain(i),1)
    if const_term in ["n","N"]:
        start_m = 1
        end_m = len(oligo.chain_sequence(1))
    elif const_term in ["c","C"]:
        start_m = mono.size() - len(oligo.chain_sequence(1)) + 1
        end_m = mono.size()
    else:
        print("const_term must be n or c")
    newpose = pyrosetta.rosetta.core.pose.Pose()
    for i in range(oligo.num_chains()):
        start_o = 1 + (i * len(oligo.chain_sequence(1)))
        end_o = ((i+1) * len(oligo.chain_sequence(1)))
        range_CA_align(mono, oligo, start_m, end_m, start_o, end_o)
        if i == 0:
            newpose = mono.clone()
        if i >= 1:
            for j in range(1,mono.num_chains()+1):
                pyrosetta.rosetta.core.pose.append_pose_to_pose(newpose,mono.split_by_chain(j),1)
        
    return newpose

def swap_chains(name,order):
    pose_in = load_pose(name)
    pose_out = pyrosetta.rosetta.core.pose.Pose()
    for i in order:
        pyrosetta.rosetta.core.pose.append_pose_to_pose(pose_out,pose_in.split_by_chain(i),1)
    return pose_out


def extend_oligomer(oligomer,extension,term,keep,oligo_hlx1,oligo_hlx2,ext_hlx1,ext_hlx2):
    oligo = load_pose(oligomer)
    ext = load_pose(extension)
    oligo_sym = oligo.num_chains()
    ext_chains = range(1,ext.num_chains()+1)
    if term in ['N','n']:
        p1 = extend_dhr(extension,oligomer,keep,ext_hlx1,ext_hlx2,oligo_hlx1,oligo_hlx2)
        p2 = oligomer_align(oligomer,p1,"c",ext_chains)
    elif term in ['C','c']:
        p1 = extend_dhr(oligomer,extension,keep,oligo_hlx1,oligo_hlx2,ext_hlx1,ext_hlx2)
        p2 = oligomer_align(oligomer,p1,"n",[x-1+oligo_sym for x in ext_chains])
    else:
        print("term must be n or c")
    return p2

def extend_oligo_hinge(hinge,parent,nrep=2,crep=2):
    par = load_pose(parent).split_by_chain(1)
    hin = load_pose(hinge)
    nres = []
    cres = []
    offset = hin.split_by_chain(1).size() - par.size()
    for i in range(1,par.size()//2 - 10):
        if par.sequence(i,i) != hin.sequence(i,i):
            nres.append(i)
    for i in range(par.size()//2 + 10,par.size()):
        if par.sequence(i,i) != hin.sequence(i+offset,i+offset):
            cres.append(i)        
    print(nres)
    print(cres)
    p1 = extend_dhr(par,hin.split_by_chain(1),"n",5,6,1,2)
    offset_n = p1.split_by_chain(1).size() - hin.split_by_chain(1).size()
    print(offset_n,offset)
    p2 = oligomer_align(hin,p1,const_term = "c")
    p3 = extend_dhr(p2.split_by_chain(1),par,"c","last","last",3,4)
    offset_c = p3.split_by_chain(1).size() - p1.split_by_chain(1).size()
    for i in nres:
        p3.replace_residue(i+offset_n,hin.residue(i),1)
    for i in cres:
        p3.replace_residue(i+offset_n+offset,hin.residue(i+offset),1)
    p4 = oligomer_align(p2,p3,const_term = "n")
    return p4

def extend_mono_hinge(hinge,parent,nrep=2,crep=2):
    par = load_pose(parent).split_by_chain(1)
    hin = load_pose(hinge)
    nres = []
    cres = []
    offset = hin.split_by_chain(1).size() - par.size()
    for i in range(1,par.size()//2 - 10):
        if par.sequence(i,i) != hin.sequence(i,i):
            nres.append(i)
    for i in range(par.size()//2 + 10,par.size()):
        if par.sequence(i,i) != hin.sequence(i+offset,i+offset):
            cres.append(i)        
    print(nres)
    print(cres)
    if nrep ==2:
        p1 = extend_dhr(par,hin.split_by_chain(1),"c",7,8,3,4)
    elif nrep ==1: 
        p1 = extend_dhr(par,hin.split_by_chain(1),"c",5,6,3,4)
    offset_n = p1.split_by_chain(1).size() - hin.split_by_chain(1).size()
    print(offset_n,offset)
    p2 = p1
    if crep == 2:
        p3 = extend_dhr(p2.split_by_chain(1),par,"c","last","last",3,4)
    elif crep == 1:
        p3 = extend_dhr(p2.split_by_chain(1),par,"c","last","last",5,6)
    offset_c = p3.split_by_chain(1).size() - p1.split_by_chain(1).size()
    for i in nres:
        p3.replace_residue(i+offset_n,hin.residue(i),1)
    for i in cres:
        p3.replace_residue(i+offset_n+offset,hin.residue(i+offset),1)
    p4 = p3
    return p4

In [4]:
p = extend_mono_hinge("in/cs_221X.pdb","in/DHR20.pdb",1,1)
p.dump_pdb("cs221_X_n1c1.pdb")
p = extend_mono_hinge("in/cs_221Y.pdb","in/DHR20.pdb",1,1)
p.dump_pdb("cs221_Y_n1c1.pdb")
p = extend_mono_hinge("in/cs_221X.pdb","in/DHR20.pdb")
p.dump_pdb("cs221_X_n2c2.pdb")
p = extend_mono_hinge("in/cs_221Y.pdb","in/DHR20.pdb")
p.dump_pdb("cs221_Y_n2c2.pdb")

[1, 2, 3, 8, 15, 16, 17, 18, 19, 24, 27, 31, 34, 35, 37, 39, 40, 42, 44, 47, 51, 53, 54, 56, 57, 62, 64]
[86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151]
38 2
[1, 2, 3, 8, 15, 16, 17, 18, 19, 24, 27, 31, 34, 35, 37, 39, 40, 42, 44, 47, 51, 53, 54, 56, 57, 62, 64]
[86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151]
38 2
[1, 2, 3, 8, 15, 16, 17, 18, 19, 24, 27, 31, 34, 35, 37, 39, 40, 42, 44, 47, 51, 53, 54, 56, 57, 62, 64]
[86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 1

True

In [4]:
p = extend_dhr('in/DHR14a.pdb','in/J7X_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7X_P_extn1.pdb')
p = extend_dhr('in/DHR14a.pdb','in/J7Y_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7Y_P_extn1.pdb')

True

In [5]:
p = extend_dhr('in/DHR14a.pdb','in/J7X_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7X_P_extn2.pdb')
p = extend_dhr('in/DHR14a.pdb','in/J7Y_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7Y_P_extn2.pdb')

True

In [6]:
p = extend_dhr('in/DHR14b.pdb','in/J7X_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7X_P_extn1_dhr14x.pdb')
p = extend_dhr('in/DHR14b.pdb','in/J7Y_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7Y_P_extn1_dhr14x.pdb')
p = extend_dhr('in/DHR14b.pdb','in/J7X_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7X_P_extn2_dhr14x.pdb')
p = extend_dhr('in/DHR14b.pdb','in/J7Y_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7Y_P_extn2_dhr14x.pdb')

True

In [3]:
p = extend_mono_hinge("in/cs201r1_X.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("cs201_X_n1c1.pdb")
p = extend_mono_hinge("in/cs201r1_Y.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("cs201_Y_n1c1.pdb")
p = extend_mono_hinge("in/cs201r1_X.pdb","in/DHR82.pdb")
p.dump_pdb("cs201_X_n2c2.pdb")
p = extend_mono_hinge("in/cs201r1_Y.pdb","in/DHR82.pdb")
p.dump_pdb("cs201_Y_n2c2.pdb")

[42, 49, 59, 62, 63, 66, 73, 77, 80, 84, 89, 94, 97, 99, 100, 104]
[127, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 160, 161, 162, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 211, 212, 213, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231]
58 -2
[42, 49, 59, 62, 63, 66, 73, 77, 80, 84, 89, 94, 97, 99, 100, 104]
[127, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 160, 161, 162, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 211, 212, 213, 216, 217, 218, 219, 220, 222, 223, 224, 225

True

In [120]:
p = extend_mono_hinge("in/pair157_232_Y_0001.pdb","in/DHR82.pdb")
p.dump_pdb("FF74_Y_n2c2_new_unrel.pdb")

[42, 49, 59, 63, 73, 80, 84, 90, 93, 96, 97, 99, 100, 101, 103, 104]
[128, 129, 130, 132, 133, 135, 136, 139, 140, 144, 159, 170]
116 3


True

In [5]:
p = extend_mono_hinge("in/pair157_232_X_0001.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("FF74_X_n1c1.pdb")
p = extend_mono_hinge("in/pair157_232_Y_0001.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("FF74_Y_n1c1.pdb")

[42, 49, 59, 63, 73, 80, 84, 90, 93, 96, 97, 99, 100, 101, 103, 104]
[128, 129, 130, 132, 133, 135, 136, 139, 140, 144, 159, 170]
58 3
[42, 49, 59, 63, 73, 80, 84, 90, 93, 96, 97, 99, 100, 101, 103, 104]
[128, 129, 130, 132, 133, 135, 136, 139, 140, 144, 159, 170]
58 3


True

In [111]:
p = extend_hinge("in/FF86-0.pdb","in/DHR21.pdb")
p.dump_pdb("v3_FF86-0_n2c2.pdb")
p = extend_hinge("in/FF86-1.pdb","in/DHR21.pdb")
p.dump_pdb("v3_FF86-1_n2c2.pdb")
p = extend_hinge("in/FF86-2.pdb","in/DHR21.pdb")
p.dump_pdb("v3_FF86-2_n2c2.pdb")

[11, 14, 19, 20, 21, 25, 27, 31, 32, 34, 35, 36, 38, 39, 40, 41, 43, 46, 47, 55, 57, 60, 62]
[88, 91, 93, 94, 95, 96, 99, 100, 101, 104, 105, 107, 108, 111, 113, 114, 115, 117, 118, 119, 120, 122, 126, 129, 130, 131, 142, 145, 146, 147, 148, 149]
76 1
[11, 14, 19, 20, 21, 25, 27, 31, 32, 34, 35, 36, 38, 39, 40, 41, 43, 46, 47, 55, 57, 60, 62]
[88, 91, 93, 94, 95, 96, 99, 100, 101, 104, 105, 107, 108, 111, 113, 114, 115, 117, 118, 119, 120, 122, 126, 129, 130, 131, 142, 145, 146, 147, 148, 149]
76 1
[11, 14, 19, 20, 21, 25, 27, 31, 32, 34, 35, 36, 38, 39, 40, 41, 43, 46, 47, 55, 57, 60, 62]
[88, 91, 93, 94, 95, 96, 99, 100, 101, 104, 105, 107, 108, 110, 111, 113, 114, 115, 117, 118, 119, 120, 122, 126, 129, 130, 131, 142, 145, 146, 147, 148, 149]
76 1


True

In [126]:
p = extend_oligo_hinge("in/FF81-0.pdb","in/DHR46.pdb")
p.dump_pdb("v3_FF81-0_n2c2.pdb")
p = extend_oligo_hinge("in/FF81-1.pdb","in/DHR46.pdb")
p.dump_pdb("v3_FF81-1_n2c2.pdb")

[20, 21, 22, 24, 27, 28, 31, 40, 43, 47, 55, 57, 58, 60, 62, 63, 64, 66, 67]
[90, 91, 94, 95, 96, 100, 101, 103, 105, 106, 107, 114, 122, 125, 126, 134, 136, 139]
78 1
[20, 21, 22, 24, 27, 28, 31, 40, 43, 47, 55, 57, 58, 59, 60, 62, 63, 64, 66, 67]
[90, 91, 94, 95, 96, 97, 100, 101, 103, 105, 106, 107, 114, 122, 125, 126, 134, 136, 139]
78 1


True

In [51]:
helix_dict(load_pose("in/pair157_232_X_0001.pdb"))

{1: [2, 28],
 2: [31, 54],
 3: [59, 86],
 4: [89, 112],
 5: [117, 147],
 6: [150, 173],
 7: [178, 205],
 8: [208, 231],
 9: [237, 262]}

In [52]:
helix_dict(load_pose("in/DHR82.pdb"))

{1: [2, 28],
 2: [31, 54],
 3: [59, 86],
 4: [89, 112],
 5: [117, 144],
 6: [147, 170],
 7: [175, 202],
 8: [205, 228]}

In [123]:
hd = helix_dict(load_pose("in/DHR46.pdb"))
for i in list(hd):
    print(hd[i][1]-hd[i][0])

14
16
15
16
15
16
15
16


In [124]:
hd = helix_dict(load_pose("in/FF81-1.pdb"))
for i in list(hd):
    print(hd[i][1]-hd[i][0])

14
16
15
14
16
16
15
16
14
16
15
14
16
16
15
16


In [2]:
pyrosetta.rosetta.protocols.protein_interface_design.FavorNativeResidue?

Docstring:     
class FavorNativeResidue changes a pose object so that its residue identities at the
initialization of FavorNativeResidue are kept in memory. If the res_type_constraint score term is set
to a value other than 0, an energy bonus will be assigned if the residue doesn't change. This is useful
e.g., in design based on a native scaffold where we want a barrier to mutation.
Init docstring:
__init__(*args, **kwargs)
Overloaded function.

1. __init__(self: pyrosetta.rosetta.protocols.protein_interface_design.FavorNativeResidue, pose: pyrosetta.rosetta.core.pose.Pose, native_residue_bonus: float) -> None

2. __init__(self: pyrosetta.rosetta.protocols.protein_interface_design.FavorNativeResidue, pose: pyrosetta.rosetta.core.pose.Pose, native_residue_bonus: pyrosetta.rosetta.utility.vector1_double) -> None

3. __init__(self: pyrosetta.rosetta.protocols.protein_interface_design.FavorNativeResidue, arg0: pyrosetta.rosetta.protocols.protein_interface_design.FavorNativeResidue) -> Non

In [3]:
p = extend_oligo_hinge("in/FF81-0.pdb","in/DHR46.pdb")

[20, 21, 22, 24, 27, 28, 31, 40, 43, 47, 55, 57, 58, 60, 62, 63, 64, 66, 67]
[90, 91, 94, 95, 96, 100, 101, 103, 105, 106, 107, 114, 122, 125, 126, 134, 136, 139]
78 1


In [6]:
res = p.residue(4)

In [13]:
res.

Docstring:
nbrs(self: pyrosetta.rosetta.core.conformation.Residue, atm: int) -> pyrosetta.rosetta.utility.vector1_unsigned_long

Convenience synonym for bonded_neighbor

C++: core::conformation::Residue::nbrs(const unsigned long) const --> const class utility::vector1<unsigned long, class std::allocator<unsigned long> > &
Type:      method


vector1_core_scoring_ScoreType[]